In [1]:
import json
from matplotlib.font_manager import json_dump
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
from datetime import datetime
import time
import seaborn as sns
import matplotlib.pylab as plt
import os
  
# Folder Path


sf_data = []
vicon_data = []
dataset_name = "run_diagonal_1"
path_sf = "/media/irfan-flw/OS/Arbeit/6GEM/Program_Development/6GEM_Dataset/sensorfloor_measurement/16092022"
file_name_sf = "sensor_floor_data_16092022_" + dataset_name + ".txt"

os.chdir(path_sf)
#file_path = f"{path}\{file_name}"
# threshold_sf_up = 1662115152.8
# threshold_sf_down = 1662115256
# threshold_vc_up = 1662115153.7486753
# threshold_vc_down = 1662115255.375461

#for line in open("../Dataset_sensor_floor/sensorfloor_measurement/16092022/sensor_floor_data_16092022_" + dataset_name + ".txt", "r"):
#for line in open("../../../media/irfan-flw/OS/Arbeit/6GEM/Program_Development/6GEM_Dataset/sensorfloor_measurement/16092022/sensor_floor_data_16092022_" + dataset_name + ".txt", "r"):
for line in open(file_name_sf, "r"):
    sf_data.append(json.loads(line))

df_sf = pd.DataFrame(sf_data)
#df_sf

**Build the dataframe of sensor floors**

In [ ]:
#Merge multiple sensor floor datasets.txt
"""
Horizontal 1: moving_horizontal_node_9, moving_horizontal_node_7
Horizontal 2: moving_horizontal_node_6, moving_horizontal_node_4
Diagonal 1: moving_diagonal_3-3_21-9, moving_diagonal_21-9_14-3
Diagonal 2: moving_diagonal_14-9_21-3, moving_diagonal_21-3_3-9
Diagonal 3: moving_diagonal_3-9_15-3, moving_diagonal_15-9_3-3, moving_diagonal_4-3_10-9
Diagonal 4: moving_diagonal_7-9_3-6, moving_diagonal_13-5_16-9, moving_diagonal_18-8_16-3
Vertical 1: moving_vertical_strip_21, moving_vertical_strip_20, moving_vertical_strip_19, moving_vertical_strip_18
Vertical 2: moving_vertical_strip_17, moving_vertical_strip_16, moving_vertical_strip_15, moving_vertical_strip_14
Vertical 3: moving_vertical_strip_13, moving_vertical_strip_12, moving_vertical_strip_11, moving_vertical_strip_10
Vertical 4: moving_vertical_strip_4, moving_vertical_strip_3
"""

In [41]:
# #1st option
# df_sf_buf = df_sf.copy()
# df_sf_buf = df_sf_buf.reindex(df_sf_buf.columns.tolist() + ['rssi','mx','my','mz'], axis=1) 

# for index, row in df_sf_buf.T.items():
#     rssi = []
#     magneto = []
#     for i in range(len(df_sf_buf['data'][index])):
#         #print((test_df['data'][index][i]['r']))
#         rssi.append(df_sf_buf['data'][index][i]['r'])
#         magneto.append(df_sf_buf['data'][index][i]['m'])
    
#     rssi_avg = np.round(np.mean(rssi, axis=0),2)
#     magneto_avg = np.round(np.mean(magneto, axis=0),2)
#     df_sf_buf.loc[index,'rssi'] = rssi_avg[0]
#     df_sf_buf.loc[index,'mx']  = magneto_avg[0]
#     df_sf_buf.loc[index,'my']  = magneto_avg[1]
#     df_sf_buf.loc[index,'mz']  = magneto_avg[2]
#     rssi.clear()
#     magneto.clear()

# df_sf_final = df_sf_buf.drop(columns=['data'])
# #df_sf_final['timestamp'] = df_sf_final['timestamp'].multiply(1000) #in ms
# df_sf_final

### Timestamp Interpolation

In [38]:
#test timestamp interpolation
#t_batch_i_now = np.zeros((23,15))
t_batch_i_old = np.zeros((23,15))
df_sf_buf = df_sf.copy()


#df_sf_buf = df_sf_buf.reindex(df_sf_buf.columns.tolist() + ['ax','ay','az','gx','gy','gz','mx','my','mz','rssi'], axis=1) 
out = []
for index, row in df_sf_buf.T.items():
    index_strip_id = int(df_sf_buf['strip_id'][index])
    index_node_id = int(df_sf_buf['node_id'][index])
    #t_batch_i_now[index_strip_id][index_node_id] = df_sf_buf['timestamp'][index]
    #print(t_batch_i_old[index_strip_id-1][index_node_id-1])
    delta_t = 0
    timestamp_i = 0

    #check timestamp
    if t_batch_i_old[index_strip_id-1][index_node_id-1] < df_sf_buf['timestamp'][index] and t_batch_i_old[index_strip_id-1][index_node_id-1] > 0:
        delta_t = (df_sf_buf['timestamp'][index] - t_batch_i_old[index_strip_id-1][index_node_id-1])/len(df_sf_buf['data'][index])
        
        for i in range(len(df_sf_buf['data'][index])):
            t_i = t_batch_i_old[index_strip_id-1][index_node_id-1] + ((1+i)*delta_t)
            if df_sf_buf['data'][index][i]['r'][0] < 0:
                #print('iter > 0: ', delta_t)
                out.append({'strip_id':df_sf_buf['strip_id'][index],'node_id':df_sf_buf['node_id'][index],
                    'ax':df_sf_buf['data'][index][i]['a'][0],'ay':df_sf_buf['data'][index][i]['a'][1],'az':df_sf_buf['data'][index][i]['a'][2],
                    'gx':df_sf_buf['data'][index][i]['g'][0],'gy':df_sf_buf['data'][index][i]['g'][1],'gz':df_sf_buf['data'][index][i]['g'][2],
                    'mx':df_sf_buf['data'][index][i]['m'][0],'my':df_sf_buf['data'][index][i]['m'][1],'mz':df_sf_buf['data'][index][i]['m'][2],
                    'r':df_sf_buf['data'][index][i]['r'][0],'timestamp':t_i})
            else:
                out.append({'strip_id':df_sf_buf['strip_id'][index],'node_id':df_sf_buf['node_id'][index],
                    'ax':df_sf_buf['data'][index][i]['a'][0],'ay':df_sf_buf['data'][index][i]['a'][1],'az':df_sf_buf['data'][index][i]['a'][2],
                    'gx':df_sf_buf['data'][index][i]['g'][0],'gy':df_sf_buf['data'][index][i]['g'][1],'gz':df_sf_buf['data'][index][i]['g'][2],
                    'mx':df_sf_buf['data'][index][i]['m'][0],'my':df_sf_buf['data'][index][i]['m'][1],'mz':df_sf_buf['data'][index][i]['m'][2],
                    'r':np.nan,'timestamp':t_i})
            t_batch_i_old[index_strip_id-1][index_node_id-1] = df_sf_buf['timestamp'][index]
    elif t_batch_i_old[index_strip_id-1][index_node_id-1] == 0:
        delta_t = 4/19
        #print('iter 0: ', delta_t)
        for i in range(len(df_sf_buf['data'][index])):
            t_i = (df_sf_buf['timestamp'][index]-4) + ((1+i)*delta_t)
            if df_sf_buf['data'][index][i]['r'][0] < 0:
                #print(t_i)
                out.append({'strip_id':df_sf_buf['strip_id'][index],'node_id':df_sf_buf['node_id'][index],
                    'ax':df_sf_buf['data'][index][i]['a'][0],'ay':df_sf_buf['data'][index][i]['a'][1],'az':df_sf_buf['data'][index][i]['a'][2],
                    'gx':df_sf_buf['data'][index][i]['g'][0],'gy':df_sf_buf['data'][index][i]['g'][1],'gz':df_sf_buf['data'][index][i]['g'][2],
                    'mx':df_sf_buf['data'][index][i]['m'][0],'my':df_sf_buf['data'][index][i]['m'][1],'mz':df_sf_buf['data'][index][i]['m'][2],
                    'r':df_sf_buf['data'][index][i]['r'][0],'timestamp':t_i})
            else:
                out.append({'strip_id':df_sf_buf['strip_id'][index],'node_id':df_sf_buf['node_id'][index],
                    'ax':df_sf_buf['data'][index][i]['a'][0],'ay':df_sf_buf['data'][index][i]['a'][1],'az':df_sf_buf['data'][index][i]['a'][2],
                    'gx':df_sf_buf['data'][index][i]['g'][0],'gy':df_sf_buf['data'][index][i]['g'][1],'gz':df_sf_buf['data'][index][i]['g'][2],
                    'mx':df_sf_buf['data'][index][i]['m'][0],'my':df_sf_buf['data'][index][i]['m'][1],'mz':df_sf_buf['data'][index][i]['m'][2],
                    'r':np.nan,'timestamp':t_i})
            t_batch_i_old[index_strip_id-1][index_node_id-1] = df_sf_buf['timestamp'][index]

df_sf_final = pd.DataFrame(out)
out.clear()
df_sf_final

,strip_id,node_id,ax,ay,az,gx,gy,gz,mx,my,mz,r,timestamp
0,13,14,0.029541,0.072998,1.007812,-0.770992,1.389313,0.213740,-49.95,-97.35,187.50,-82.0,1.663338e+09
1,13,14,0.032471,0.074951,0.999268,-0.496183,1.351145,-0.038168,-50.40,-98.85,188.40,-83.0,1.663338e+09
2,13,14,0.029297,0.076172,0.999023,-0.694656,1.480916,0.145038,-48.90,-98.40,188.40,-83.0,1.663338e+09
3,13,14,0.028320,0.069824,1.002197,-0.458015,1.175573,0.145038,-50.10,-97.95,187.95,-84.0,1.663338e+09
4,13,14,0.029297,0.074951,1.002930,-0.595420,1.198473,0.137405,-48.15,-97.80,188.10,-83.0,1.663338e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...
291410,17,15,0.039795,0.079590,1.007324,-0.106870,0.854962,0.648855,-4.35,36.60,1.65,-80.0,1.663338e+09
291411,17,15,0.036865,0.081787,1.006348,-0.152672,0.938931,0.412214,-5.70,36.60,0.30,-80.0,1.663338e+09
291412,17,15,0.041504,0.080566,1.006836,-0.076336,0.900763,0.572519,-6.45,36.30,1.35,-80.0,1.663338e+09
291413,17,15,0.036377,0.078857,1.003906,-0.221374,1.045802,0.709924,-5.10,37.05,1.65,-80.0,1.663338e+09


In [43]:
# #df_strip_node_id = pd.DataFrame(columns=['strip_id','node_id','timestamp'])
# df_strip_node_id = df_sf_final[['strip_id','node_id','timestamp']].copy()
# df_strip_node_id['strip_id'] = pd.to_numeric(df_strip_node_id['strip_id'])
# df_strip_node_id['node_id'] = pd.to_numeric(df_strip_node_id['node_id'])
# df_strip_node_id['time'] = pd.to_datetime(df_strip_node_id['timestamp'],unit='s')
# df_strip_node_id = df_strip_node_id.set_index('time')
# # #df_sf_resample
# df_strip_node_id = df_strip_node_id.resample('40ms').mean().ffill()
# df_strip_node_id

In [40]:
# df_sf_resample = df_sf_final.copy()
# df_sf_resample['strip_id'] = pd.to_numeric(df_sf_resample['strip_id'])
# df_sf_resample['node_id'] = pd.to_numeric(df_sf_resample['node_id'])
# df_sf_resample['time'] = pd.to_datetime(df_sf_resample['timestamp'],unit='s')
# df_sf_resample = pd.melt(df_sf_resample, id_vars=['strip_id','node_id','ax','ay','az','gx','gy','gz','mx','my','mz','r','timestamp'], value_name='time').drop('variable', axis=1)
# df_sf_resample = df_sf_resample.set_index('time')
# # #df_sf_resample
# df_sf_final = df_sf_resample.resample('25ms').mean()
# # df_sf_final = df_sf_final.dropna()
# # # df_sf_final['timestamp'] = df_sf_final.index.astype(np.int64) / int(1e9)
# df_sf_final

/tmp/ipykernel_7123/599115895.py:5: FutureWarning: This dataframe has a column name that matches the 'value_name' column name of the resulting Dataframe. In the future this will raise an error, please set the 'value_name' parameter of DataFrame.melt to a unique name.
  df_sf_resample = pd.melt(df_sf_resample, id_vars=['strip_id','node_id','ax','ay','az','gx','gy','gz','mx','my','mz','r','timestamp'], value_name='time').drop('variable', axis=1)


,strip_id,node_id,ax,ay,az,gx,gy,gz,mx,my,mz,r,timestamp
time,,,,,,,,,,,,,
2022-09-16 14:14:21.575,13.000000,14.000000,0.029541,0.072998,1.007812,-0.770992,1.389313,0.213740,-49.950,-97.350,187.500,-82.0,1.663338e+09
2022-09-16 14:14:21.600,15.000000,13.000000,0.104248,0.101074,0.996582,0.064885,0.022901,1.267176,63.750,-65.775,109.875,NaN,1.663338e+09
2022-09-16 14:14:21.625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-16 14:14:21.650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-16 14:14:21.675,9.166667,7.333333,0.076009,0.162760,0.994832,-0.790076,0.819338,0.866412,19.525,-33.375,147.575,-83.8,1.663338e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-16 14:17:44.925,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-16 14:17:44.950,7.333333,6.333333,0.060384,0.139160,1.008952,-1.854962,0.114504,0.170483,-19.400,-31.000,210.100,-86.5,1.663338e+09
2022-09-16 14:17:44.975,12.500000,8.500000,0.051636,0.132202,0.963745,-1.633588,0.152672,0.664122,-75.675,-31.200,78.975,-95.0,1.663338e+09


In [4]:
# df_sf_final.to_csv('sf_data_run1_diagonal_resample.csv',index=False)

In [24]:
# df_sf_resample = df_sf_final.copy()
# df_sf_resample['time'] = pd.to_datetime(df_sf_resample['timestamp'],unit='s')
# df_sf_final = pd.melt(df_sf_resample, id_vars=['strip_id','node_id','ax','ay','az','gx','gy','gz','mx','my','mz','r','timestamp'], value_name='time').drop('variable', axis=1)
# df_sf_final = df_sf_final.groupby(['strip_id','node_id']).apply(lambda x : x.drop_duplicates(['strip_id','node_id','ax','ay','az','gx','gy','gz','mx','my','mz','r','timestamp'])
#                                           .set_index('time')
#                                           .resample('50ms')
#                                           .mean())
# df_sf_final

/tmp/ipykernel_7123/1995645911.py:3: FutureWarning: This dataframe has a column name that matches the 'value_name' column name of the resulting Dataframe. In the future this will raise an error, please set the 'value_name' parameter of DataFrame.melt to a unique name.
  df_sf_final = pd.melt(df_sf_resample, id_vars=['strip_id','node_id','ax','ay','az','gx','gy','gz','mx','my','mz','r','timestamp'], value_name='time').drop('variable', axis=1)


ax        ay        az  \
strip_id node_id time                                                    
1        1       2022-09-16 14:14:25.150  0.047363  0.174805  0.988281   
                 2022-09-16 14:14:25.200       NaN       NaN       NaN   
                 2022-09-16 14:14:25.250       NaN       NaN       NaN   
                 2022-09-16 14:14:25.300       NaN       NaN       NaN   
                 2022-09-16 14:14:25.350       NaN       NaN       NaN   
...                                            ...       ...       ...   
9        9       2022-09-16 14:17:42.650       NaN       NaN       NaN   
                 2022-09-16 14:17:42.700       NaN       NaN       NaN   
                 2022-09-16 14:17:42.750       NaN       NaN       NaN   
                 2022-09-16 14:17:42.800       NaN       NaN       NaN   
                 2022-09-16 14:17:42.850  0.030273  0.122803  1.016602   

                                                gx        gy        gz  \
strip_id node_id time                                                    
1        1       2022-09-16 14:14:25.150 -1.198473 -0.076336  1.793893   
                 2022-09-16 14:14:25.200       NaN       NaN       NaN   
                 2022-09-16 14:14:25.250       NaN       NaN       NaN   
                 2022-09-16 14:14:25.300       NaN       NaN       NaN   
                 2022-09-16 14:14:25.350       NaN       NaN       NaN   
...                                            ...       ...       ...   
9        9       2022-09-16 14:17:42.650       NaN       NaN       NaN   
                 2022-09-16 14:17:42.700       NaN       NaN       NaN   
                 2022-09-16 14:17:42.750       NaN       NaN       NaN   
                 2022-09-16 14:17:42.800       NaN       NaN       NaN   
                 2022-09-16 14:17:42.850 -0.335878  0.473282  2.740458   

                                              mx      my     mz     r  \
strip_id node_id time                                                   
1        1       2022-09-16 14:14:25.150  171.15  -67.95  52.50 -87.0   
                 2022-09-16 14:14:25.200     NaN     NaN    NaN   NaN   
                 2022-09-16 14:14:25.250     NaN     NaN    NaN   NaN   
                 2022-09-16 14:14:25.300     NaN     NaN    NaN   NaN   
                 2022-09-16 14:14:25.350     NaN     NaN    NaN   NaN   
...                                          ...     ...    ...   ...   
9        9       2022-09-16 14:17:42.650     NaN     NaN    NaN   NaN   
                 2022-09-16 14:17:42.700     NaN     NaN    NaN   NaN   
                 2022-09-16 14:17:42.750     NaN     NaN    NaN   NaN   
                 2022-09-16 14:17:42.800     NaN     NaN    NaN   NaN   
                 2022-09-16 14:17:42.850  -72.00 -156.75 -95.85   NaN   

                                             timestamp  
strip_id node_id time                                   
1        1       2022-09-16 14:14:25.150  1.663338e+09  
                 2022-09-16 14:14:25.200           NaN  
                 2022-09-16 14:14:25.250           NaN  
                 2022-09-16 14:14:25.300           NaN  
                 2022-09-16 14:14:25.350           NaN  
...                                                ...  
9        9       2022-09-16 14:17:42.650           NaN  
                 2022-09-16 14:17:42.700           NaN  
                 2022-09-16 14:17:42.750           NaN  
                 2022-09-16 14:17:42.800           NaN  
                 2022-09-16 14:17:42.850  1.663338e+09  

[1358230 rows x 11 columns]

In [25]:
# # the inverse of groupby, reset_index
# df_sf_clean = df_sf_final.reset_index()

# # set again the timestamp as index
# df_sf_clean = df_sf_clean.set_index('time')

# df_sf_clean = df_sf_clean.sort_values('timestamp')
# df_sf_clean = df_sf_clean.dropna()
# df_sf_clean

,strip_id,node_id,ax,ay,az,gx,gy,gz,mx,my,mz,r,timestamp
time,,,,,,,,,,,,,
2022-09-16 14:14:21.550,13,14,0.029541,0.072998,1.007812,-0.770992,1.389313,0.213740,-49.95,-97.35,187.50,-82.0,1.663338e+09
2022-09-16 14:14:21.650,10,10,0.106689,0.180420,0.972656,0.152672,1.648855,1.549618,-140.25,-162.15,209.40,-86.0,1.663338e+09
2022-09-16 14:14:21.650,11,1,0.161133,0.178467,0.982910,-2.022901,0.007634,0.198473,112.35,-75.75,80.40,-80.0,1.663338e+09
2022-09-16 14:14:21.650,2,13,0.057129,0.143311,0.960938,0.916031,0.442748,0.213740,-156.00,28.05,238.95,-86.0,1.663338e+09
2022-09-16 14:14:21.650,15,13,0.026611,0.170410,1.011963,-0.557252,0.473282,0.763359,28.20,-29.25,93.60,-89.0,1.663338e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-16 14:17:44.950,6,13,0.048584,0.087891,0.989502,-3.007634,0.335878,1.137405,55.65,-8.55,152.70,-95.0,1.663338e+09
2022-09-16 14:17:45.000,17,15,0.041260,0.077881,1.011719,-0.244275,0.961832,0.564885,-7.95,35.85,0.60,-80.0,1.663338e+09
2022-09-16 14:17:45.000,2,15,0.070557,0.109863,0.992188,-3.450382,0.732824,2.961832,-53.10,39.00,-55.20,-74.0,1.663338e+09


In [21]:
# df_sf_clean.to_csv('sf_data_run1_diagonal_resample.csv',index=False)

In [ ]:
# #df_sf_resample = df_sf_resample.set_index('timestamp')
# df_sf_final = df_sf_resample.resample('45ms').mean()
# #df_sf_final = df_sf_final.dropna()
# #df_sf_final['timestamp'] = df_sf_final.index.astype(np.int64) / int(1e9)
# df_sf_final

**Build Dataset of VICON Coordinates**

In [ ]:
#DATASET OF VICON COORDINATES
#for line in open("../Dataset_sensor_floor/vicon_measurement/16092022/vicon_data_16092022_" + dataset_name + ".txt", "r"):
path_vicon = "/media/irfan-flw/OS/Arbeit/6GEM/Program_Development/6GEM_Dataset/vicon_measurement/16092022"
file_name_vicon = "vicon_data_16092022_" + dataset_name + ".txt"

os.chdir(path_vicon)

for line in open(file_name_vicon, "r"):
    vicon_data.append(json.loads(line))

df_vicon = pd.DataFrame(vicon_data)
#df_vicon['time'] = pd.to_datetime(df_vicon['time'],unit='s')
df_vicon.shape

In [ ]:
df_vicon_buf = df_vicon.copy()
df_vicon_buf = df_vicon_buf.reindex(df_vicon_buf.columns.tolist() + ['X','Y'], axis=1) 

for index, row in df_vicon_buf.T.items():
    for i in range(len(df_vicon_buf['translation'][index])):
        #print((test_df['data'][index][i]['r']))
        df_vicon_buf.loc[index,'X'] = df_vicon_buf['translation'][index][0]
        df_vicon_buf.loc[index,'Y'] = df_vicon_buf['translation'][index][1]

df_vicon_buf = df_vicon_buf.drop(columns=['object','translation','rotation'])
df_vicon_buf

***Filter and Resample Vicon Data***

In [ ]:

threshold_sf_up = 1663337798.499045
threshold_sf_low = 1663337796.8056197
df_vicon_buf = df_vicon_buf[(df_vicon_buf['time'] < threshold_sf_low) | (df_vicon_buf['time'] > threshold_sf_up)]
# # df_sf_filter = df_sf_final[(df_sf_final['timestamp'] < threshold_sf_down)]
df_vicon_buf

In [ ]:
df_vicon_resample = df_vicon_buf.copy()
df_vicon_resample['time'] = pd.to_datetime(df_vicon_resample['time'],unit='s')
#df_vicon_resample['time'] = df_vicon_resample['time'].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S.%f'))
df_vicon_resample = df_vicon_resample.set_index('time')
df_vicon_resample

In [ ]:
# df_vicon_resample = df_vicon_final.copy()
# df_vicon_resample['time'] = pd.to_datetime(df_vicon_resample['time'],unit='s')
df_vicon_final = df_vicon_resample.resample('50ms').ffill()
df_vicon_final = df_vicon_final.dropna()
df_vicon_final

In [ ]:
df_vicon_final['time'] = df_vicon_final.index.astype(np.int64) / int(1e9)
df_vicon_final

In [ ]:
df_vicon_final.to_csv('vicon_data_run1_diagonal_resample.csv')

**MERGE RSSI HEATMAP & VICON COORDINATES WITH ANIMATION**

In [ ]:
# #MERGE RSSI HEATMAP & VICON COORDINATES WITH ANIMATION

# #Heatmap animation over the time
# from turtle import width
# import matplotlib.animation as animation
# %matplotlib ipympl
# from matplotlib.widgets import Slider, Button, RadioButtons
# from mpl_toolkits.axes_grid1.inset_locator import inset_axes

# #DEFINE DATASET ARRAY
# x_sf = df_sf_filter['strip_id'].to_numpy()
# y_sf = df_sf_filter['node_id'].to_numpy()
# z_sf = df_sf_filter['rssi'].to_numpy()
# t_sf = df_sf_filter['timestamp'].to_numpy()
# dataSet_sf = np.array([x_sf, y_sf, z_sf, t_sf])
# numDataPoints_sf = len(t_sf)
# num_of_nodes = 15
# num_of_strips = 23


# #CREATE 3D ARRAY FOR HEATMAP
# rssi_mat = np.empty((numDataPoints_sf,15,23))
# timestamp_arr = np.empty((numDataPoints_sf,15,23))
# fig = plt.figure(figsize=(15, 9))

# for index in range(numDataPoints_sf):
#     for node in range(num_of_nodes):
#         for strip in range(num_of_strips):
#             if node == int(dataSet_sf[1][index])-1 and strip == int(dataSet_sf[0][index])-1:
#                 rssi_mat[index][int(dataSet_sf[1][index])-1][int(dataSet_sf[0][index])-1] = (dataSet_sf[2][index])
#                 timestamp_arr[index][int(dataSet_sf[1][index])-1][int(dataSet_sf[0][index])-1] = (dataSet_sf[3][index])
#             else:
#                 rssi_mat[index][node][strip] = rssi_mat[index-1][node][strip]
#                 timestamp_arr[index][node][strip] = timestamp_arr[index-1][node][strip]


# numDataPoints_sf = dataSet_sf.shape[1]

# delta_t_sf = round((t_sf.max()-t_sf.min())/numDataPoints_sf, 6)

# #-------------------------------------------------------------------------------------------------------------------#

# #CREATE DATASET FOR VICON COORDINATES
# t_vc = df_vicon_filter['time'].to_numpy()
# x_vc = df_vicon_filter['X'].to_numpy()
# y_vc = df_vicon_filter['Y'].to_numpy()
# dataSet_vc = np.array([x_vc, y_vc, t_vc])
# numDataPoints_vc = len(t_vc)


# numDataPoints_vc = dataSet_vc.shape[1]

# delta_t_vc = round((t_vc.max()-t_vc.min())/numDataPoints_vc, 6)
# #-------------------------------------------------------------------------------------------------------------------#

# # GET SOME MATPLOTLIB OBJECTS
# #fig, (ax1, ax2) = plt.subplots(figsize=(10,12), nrows=2)
# fig = plt.figure(figsize=(12,14))
# #plt.subplots_adjust(bottom=0.25)


# # AXES PROPERTIES RSSI HEATMAP
# ax1 = fig.add_subplot(211) #(row, column, pos)
# #ax1.set_autoscale_on
# ax1.set_xlabel('Strip ID')
# ax1.set_ylabel('Node ID')
# ax1.set_title('RSSI Heatmap')

# # # AXES PROPERTIES VICON COORDINATES
# ax2 = fig.add_subplot(212)
# ax2.set_xlim(-11.185, 10.185)
# ax2.set_ylim(-6.425, 7.575)
# ax2.set_xlabel('X(t)')
# ax2.set_ylabel('Y(t)')
# ax2.set_title('Trajectory of robot')


# #DISPLAY INITIAL IMAGE
# im_h = ax1.imshow(rssi_mat[numDataPoints_sf-1], cmap="YlGnBu", aspect='auto')
# axins = inset_axes(ax1,
#                    width="1%",  # width = 5% of parent_bbox width
#                    height="90%",  # height : 50%
#                    loc='lower left',
#                    bbox_to_anchor=(1.05, 1.25, 1, 1),
#                    bbox_transform=ax2.transAxes,
#                    borderpad=0,
#                    )
# cbar = plt.colorbar(im_h, cax=axins, ax=ax1)
# cbar.set_label(label='RSSI (dBm)', size=15)
# cbar.ax.tick_params(labelsize=12)

# line = ax2.plot(dataSet_vc[0], dataSet_vc[1], lw=2, c='g')[0] # For line plot


# def find_nearest(array, value):
#     array = np.asarray(array)
#     idx = (np.abs(array - value)).argmin()
#     return idx

# def animate(num, rssi_mat, line, dataSet_sf, dataSet_vc):
#     line.set_xdata(dataSet_vc[0, :num])
#     line.set_ydata(dataSet_vc[1, :num])
#     index_rssi = find_nearest(dataSet_sf[3], dataSet_vc[2,num])
#     im_h.set_data(rssi_mat[index_rssi]) 
#     return line

# plt.subplots_adjust(left=0.1, bottom=0.2, right=0.88, top=0.97, hspace=0.2)
# anim = animation.FuncAnimation(fig, animate, frames=numDataPoints_vc, fargs=(rssi_mat, line, dataSet_sf, dataSet_vc), interval=1, repeat=False, blit=False)
# # anim.save(r'Animation_Move_Around4_17082022.mp4')

# plt.show()

**MERGE RSSI HEATMAP & VICON COORDINATES WITH SLIDER**

In [ ]:
#MERGE RSSI HEATMAP & VICON COORDINATES WITH SLIDER

#Heatmap animation over the time
from turtle import width
import matplotlib.animation as animation
%matplotlib ipympl
from matplotlib.widgets import Slider, Button, RadioButtons
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

#DEFINE DATASET ARRAY
x_sf = df_sf_final['strip_id'].to_numpy()
y_sf = df_sf_final['node_id'].to_numpy()
z_sf = df_sf_final['r'].to_numpy()
t_sf = df_sf_final['timestamp'].to_numpy()
dataSet_sf = np.array([x_sf, y_sf, z_sf, t_sf])
numDataPoints_sf = len(t_sf)
num_of_nodes = 15
num_of_strips = 23


#CREATE 3D ARRAY FOR HEATMAP
rssi_mat = np.empty((numDataPoints_sf,15,23))
timestamp_arr = np.empty((numDataPoints_sf,15,23))
fig = plt.figure(figsize=(15, 9))

for index in range(numDataPoints_sf):
    for node in range(num_of_nodes):
        for strip in range(num_of_strips):
            if node == int(dataSet_sf[1][index])-1 and strip == int(dataSet_sf[0][index])-1:
                rssi_mat[index][int(dataSet_sf[1][index])-1][int(dataSet_sf[0][index])-1] = (dataSet_sf[2][index])
                timestamp_arr[index][int(dataSet_sf[1][index])-1][int(dataSet_sf[0][index])-1] = (dataSet_sf[3][index])
            else:
                rssi_mat[index][node][strip] = rssi_mat[index-1][node][strip]
                timestamp_arr[index][node][strip] = timestamp_arr[index-1][node][strip]


numDataPoints_sf = dataSet_sf.shape[1]
# print("dataset sf:",dataSet_sf.shape, "rssi new: ", rssi_mat.shape)
# print(dataSet_sf)

delta_t_sf = round((t_sf.max()-t_sf.min())/numDataPoints_sf, 6)

#print("new t_sf:", t_sf, t_sf.shape, "x_sf:", x_sf.shape, "y_sf:", y_sf.shape, "z_sf:", z_sf.shape)

#-------------------------------------------------------------------------------------------------------------------#

#CREATE DATASET FOR VICON COORDINATES
t_vc = df_vicon_final['time'].to_numpy()
x_vc = df_vicon_final['X'].to_numpy()
y_vc = df_vicon_final['Y'].to_numpy()
dataSet_vc = np.array([x_vc, y_vc, t_vc])
numDataPoints_vc = len(t_vc)

print("dataset vc old:", dataSet_vc.shape)

numDataPoints_vc = dataSet_vc.shape[1]
# print("dataset vc:",dataSet_vc.shape)
# print(dataSet_vc)

delta_t_vc = round((t_vc.max()-t_vc.min())/numDataPoints_vc, 6)
#-------------------------------------------------------------------------------------------------------------------#

# GET SOME MATPLOTLIB OBJECTS
#fig, (ax1, ax2) = plt.subplots(figsize=(10,12), nrows=2)
fig = plt.figure(figsize=(12,14))
plt.subplots_adjust(bottom=0.25)


# AXES PROPERTIES RSSI HEATMAP
ax1 = fig.add_subplot(211) #(row, column, pos)
#ax1.set_autoscale_on
ax1.set_xlabel('Strip ID')
ax1.set_ylabel('Node ID')
ax1.set_title('RSSI Heatmap')

# # AXES PROPERTIES VICON COORDINATES
ax2 = fig.add_subplot(212)
ax2.set_xlim(-11.185, 10.185)
ax2.set_ylim(-6.425, 7.575)
ax2.set_xlabel('X(t)')
ax2.set_ylabel('Y(t)')
ax2.set_title('Trajectory of robot')


# Defining the Slider button
# xposition, yposition, width and height
ax_slide_sf = plt.axes([0.155, 0.02, 0.65, 0.03])
ax_slide_vc = plt.axes([0.155, 0.07, 0.65, 0.03])
ax_slide_merge = plt.axes([0.155, 0.12, 0.65, 0.03])


#DISPLAY INITIAL IMAGE
im_h = ax1.imshow(rssi_mat[numDataPoints_sf-1], cmap="YlGnBu", aspect='auto')
axins = inset_axes(ax1,
                   width="1%",  # width = 5% of parent_bbox width
                   height="90%",  # height : 50%
                   loc='lower left',
                   bbox_to_anchor=(1.05, 1.25, 1, 1),
                   bbox_transform=ax2.transAxes,
                   borderpad=0,
                   )
cbar = plt.colorbar(im_h, cax=axins, ax=ax1)
cbar.set_label(label='RSSI (dBm)', size=15)
cbar.ax.tick_params(labelsize=12)


#cbar.ax.set_axes_locator
# #cbar.set_ticks()

line = ax2.plot(dataSet_vc[0], dataSet_vc[1], lw=2, c='g')[0] # For line plot


# Properties of the slider

#normal data
timestamp_rssi = Slider(ax_slide_sf, 'Timestamp SF(s)',
                  t_sf.min(), t_sf.max(), valinit=t_sf.min(), valstep=delta_t_sf)
timestamp_vc = Slider(ax_slide_vc, 'Timestamp VC(s)',
                  t_vc.min(), t_vc.max(), valinit=t_vc.min(), valstep=delta_t_vc)
timestamp_merge = Slider(ax_slide_merge, 'Timestamp (s)',
                  t_vc.min(), t_vc.max(), valinit=t_vc.min(), valstep=delta_t_vc)

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx


def update_heatmap(val):
    pos = timestamp_rssi.val
    index = find_nearest(dataSet_sf[3], pos)
    im_h.set_data(rssi_mat[index])

def update_coord(val):
    pos = timestamp_vc.val
    index = find_nearest(dataSet_vc[2], pos)
    line.set_xdata(dataSet_vc[0, :index])
    line.set_ydata(dataSet_vc[1, :index])

    #redrawing the figure
    fig.canvas.draw()    

def update_all(val):
    pos = timestamp_merge.val
    print(pos)
    index = find_nearest(dataSet_vc[2], pos)
    line.set_xdata(dataSet_vc[0, :index])
    line.set_ydata(dataSet_vc[1, :index])

    index2 = find_nearest(dataSet_sf[3], pos)
    im_h.set_data(rssi_mat[index2])

    #redrawing the figure
    fig.canvas.draw() 

# Calling the function "update" when the value of the slider is changed
timestamp_rssi.on_changed(update_heatmap)
timestamp_vc.on_changed(update_coord)
timestamp_merge.on_changed(update_all)

#fig.tight_layout()
plt.subplots_adjust(left=0.1, bottom=0.2, right=0.88, top=0.97, hspace=0.2)
#plt.subplots_adjust(left=0.155, bottom=0.2, right=0.91, top=0.97, hspace=0.2)
#plt.subplot_tool()
plt.show()

In [ ]:
# threshold_sf_up = 1663337798.499045
# threshold_sf_low = 1663337796.8056197

In [ ]:
# plt.savefig('../Dataset_sensor_floor/Dataset_Final/16092022/' + dataset_name + '.png', dpi=300)

**CREATE FRAME FOR MERGED DATASET**

In [ ]:
#CREATE ARRAY FOR DATA SENSORS
from tqdm import tqdm_notebook as tqdm


path = "/home/irfan-flw/6GEM/Coding/Vicon_Pos_Converter"

os.chdir(path)

KEYS = ['ax', 'ay', 'az', 'gx', 'gy', 'gz', 'mx', 'my', 'mz', 'r', 'timestamp']
Vicon_Coords = pd.read_csv("vicon_node_positions.csv")
df_data = Vicon_Coords[['strip_id','node_id']]
df_data = df_data.astype('int32')

#CREATE 4D ARRAY

X = np.zeros([23, 15, 11])


# init_frame()
def generate_frame(X):
    X_df = X.reshape([345,11])
    df_X = pd.DataFrame(X_df, columns=KEYS)
    df_data[['ax', 'ay', 'az', 'gx', 'gy', 'gz', 'mx', 'my', 'mz', 'r', 'timestamp']] = df_X
    #df_X[['strip_id','node_id']] = strip_node_df

    #print(df_data)
    frame_i = df_data.to_json(orient='columns')
    #print(frame_i)

    return frame_i


def get_data_from_data_frame(df_data):
    #X = np.zeros([23, 15, 11])
    #frame = {}
    for i, key in enumerate(KEYS):
        X[int(df_data.strip_id) - 1, int(df_data.node_id) - 1, i] = df_data[key]
    
    frame = generate_frame(X)
    # X_df = X_df.reshape([345,11])
    # df_X = pd.DataFrame(X_df, columns=KEYS)
    #print(df_X)
    t = df_data.timestamp

    return X, t, frame

def build_sensor_array(df):

    X = np.zeros([len(df), 23, 15, 11])
    t = np.zeros([len(df), 1])
    frames = []

    # Generate a single frame for each row
    for index, row in tqdm(df.T.items(), total=len(df)):

        df_i = row
        
        X_i, t_i, frame_i = get_data_from_data_frame(df_i)
        X[index] = X_i
        t[index] = t_i
        frames.append(frame_i)
        #print(X[index])
    
    return X, t, frames

sensor_data, t_data, frames = build_sensor_array(df_sf_final)

print(sensor_data.shape, t_data.shape, len(frames))

**Check index of timestamp threshold**

In [ ]:
def closest_timestamp(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

index_low_X = closest_timestamp(t_data, threshold_sf_low)
index_low_vc = closest_timestamp(dataSet_vc[2], threshold_sf_low)
index_up_X = closest_timestamp(t_data, threshold_sf_up)
index_up_vc = closest_timestamp(dataSet_vc[2], threshold_sf_up)

print(index_low_X,index_up_X, index_low_vc,index_up_vc)

#Filter out the array
sensor_data_final = sensor_data[index_low_X:index_up_X]
t_data_final = t_data[index_low_X:index_up_X]
frames_final = frames[index_low_X:index_up_X]
dataSet_vc_final = dataSet_vc[:,index_low_vc:index_up_vc]
dataSet_vc_final = np.transpose(dataSet_vc_final)

print(sensor_data_final.shape, t_data_final.shape, len(frames_final), dataSet_vc_final.shape)

In [ ]:
#MERGE DATASET
frame_number = np.empty(dataSet_vc_final.shape[0], dtype=object)
frame_data = np.empty(dataSet_vc_final.shape[0], dtype=object)
frame_count = 0
for row in range(dataSet_vc_final.shape[0]):
    time_vc = dataSet_vc[2][row]
    #index_vc = closest_timestamp(dataSet_vc[2], time_vc)
    index_sf = closest_timestamp(t_data_final, time_vc)

    frame_number[row] = str(frame_count)
    frame_data[row] = frames_final[index_sf]
    frame_count += 1

**GENERATES THE DATASET AS .CSV FILE**

In [ ]:
array_frame_vc = dataSet_vc_final.astype(object)

dataSet_final = np.array([frame_number, array_frame_vc[:,0], array_frame_vc[:,1], frame_data])
dataSet_final = np.transpose(dataSet_final)
#dataSet_final.shape
frame_sf = pd.DataFrame(dataSet_final, columns=['frame_number','vicon_x','vicon_y','data'])
data_dir = '../Dataset_sensor_floor/Dataset_Final/02092022/'
csv_name = data_dir + 'dataset_0209022_' + dataset_name + '.csv'
frame_sf.to_csv(csv_name, index=False)
#dataSet_final

In [ ]:
frame_sf